In [1]:
import pandas as pd
import numpy as np
import io
import gdreg

%load_ext autoreload
%autoreload 2

In [46]:
def read_vep(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        sep='\t'
    ).rename(columns={'#Uploaded_variation': 'SNP'})

# Read annotation
PGEN_FILE = '/n/scratch3/users/j/jz286/imp_geno/ukb_imp_chr@_v3'
VEP_FILE = "/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/baseline_annot/vep/ukb_imp_chr@_v3.vep_annot"
OUT_FILE = "/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/baseline_annot/vep/ukb_imp_chr@_v3.vep.annot.gz"

for CHR in range(1,23):
# for CHR in [22]:
    print("# CHR %d" % CHR)
    df_snp_chr = gdreg.util.read_pgen(PGEN_FILE.replace('@', '%d' % CHR))['pvar']
    df_vep = read_vep(VEP_FILE.replace("@", "%d"%CHR))
    
    df_annot = df_snp_chr[['CHR', 'SNP', 'BP']]
    for col in ['Gene', 'DOMAINS']:
        temp_dic = {x:y for x,y in zip(df_vep['SNP'], df_vep[col]) if y!='-'}
        df_annot[col] = [temp_dic[x] if x in temp_dic else '' for x in df_annot['SNP']]
    df_annot = df_annot.loc[df_annot['DOMAINS']!='']
    df_annot.columns = ['CHR', 'SNP', 'BP', 'AN:GENE', 'AN:DOMAINS']
    gdreg.util.write_annot(df_annot, OUT_FILE.replace("@", "%d"%CHR))

# CHR 1


/home/jz286/myenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (15,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/jz286/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 2


/home/jz286/myenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 3
'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 4
'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 5
'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 6
'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 7
'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 8
'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 9
'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 10
'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 11
'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 12
'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 13
'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 14
'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 15
'CM' missing from df_annot.columns, add 'CM' column with 0
# CHR 16
'CM' missing from df_annot.columns, add 'CM' column wit

### Explore

In [171]:
def read_vep(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        sep='\t'
    ).rename(columns={'#Uploaded_variation': 'SNP'})

# Read annotation
PGEN_FILE = '/n/scratch3/users/j/jz286/imp_geno/ukb_imp_chr@_v3'
VEP_FILE = "/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/baseline_annot/vep/ukb_imp_chr@_v3.vep_annot"

df_annot_list = []
for CHR in range(1,23):
# for CHR in [21, 22]:
    print("# CHR %d" % CHR)
    df_snp_chr = gdreg.util.read_pgen(PGEN_FILE.replace('@', '%d' % CHR))['pvar']
    df_snp_chr['MAF'] = gdreg.util.read_pgen(
        PGEN_FILE.replace("@", "%d" % CHR))["afreq"]["MAF"].astype(np.float32)
    df_vep = read_vep(VEP_FILE.replace("@", "%d"%CHR))
    
    df_annot = df_snp_chr[['CHR', 'SNP', 'BP', 'MAF']].copy()
    for col in ['Gene', 'Consequence', 'IMPACT', 'SIFT', 'PolyPhen', 'DOMAINS']:
        temp_dic = {x:y for x,y in zip(df_vep['SNP'], df_vep[col]) if y!='-'}
        df_annot[col] = [temp_dic[x] if x in temp_dic else '' for x in df_annot['SNP']]
        
    df_annot['SIFT_num'] = [
    float(x.split('(')[1].replace(')', '')) if x!='' else np.nan for x in df_annot['SIFT']]
    df_annot['SIFT_cate'] = [
        x.split('(')[0] if x!='' else np.nan for x in df_annot['SIFT']]
    df_annot['PolyPhen_num'] = [
        float(x.split('(')[1].replace(')', '')) if x!='' else np.nan for x in df_annot['PolyPhen']]
    df_annot['PolyPhen_cate'] = [x.split('(')[0] if x!='' else np.nan for x in df_annot['PolyPhen']]
    df_annot = df_annot.loc[df_annot['DOMAINS']!='']
    df_annot_list.append(df_annot.copy())

df_annot = pd.concat(df_annot_list, axis=0)
df_annot.index = df_annot['SNP']

# CHR 1


/home/jz286/myenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (15,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


# CHR 2


/home/jz286/myenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


# CHR 3
# CHR 4
# CHR 5
# CHR 6
# CHR 7
# CHR 8
# CHR 9
# CHR 10
# CHR 11
# CHR 12
# CHR 13
# CHR 14
# CHR 15
# CHR 16
# CHR 17
# CHR 18
# CHR 19
# CHR 20
# CHR 21
# CHR 22


In [172]:
dic_mbin = {'common' : [0.05, 1], 'lf' : [0.005, 0.05]}
for col in ['Consequence', 'IMPACT', 'SIFT_cate', 'PolyPhen_cate']:
    print(col)
    temp_df = df_annot.loc[df_annot['DOMAINS']!=''].copy()
    temp_df_ct = temp_df.groupby(col).agg({'SNP':len}).sort_values('SNP', ascending=False)
    temp_df_ct['SNP'] = temp_df_ct['SNP'] / temp_df_ct['SNP'].sum()
    cate_list = list(temp_df_ct.index[temp_df_ct['SNP']>0.01])
    
    ct_dic = {}
    for mbin in dic_mbin: 
        maf_lb,maf_ub = dic_mbin[mbin]
        ind_maf = (df_annot['MAF']>=maf_lb) & (df_annot['MAF']<maf_ub)
        temp_df = df_annot.loc[(df_annot['DOMAINS']!='') & ind_maf].copy()
        ct_dic[mbin] = temp_df.groupby(col).agg({'SNP':len}).sort_values('SNP', ascending=False)
        
    for cate in cate_list:
        print('%-40s \t common %0.3f (%5d/%5d) \t lf %0.3f (%d/%d)' % (
            cate, 
            ct_dic['common'].loc[cate, 'SNP'] / ct_dic['common']['SNP'].sum(),
            ct_dic['common'].loc[cate, 'SNP'], ct_dic['common']['SNP'].sum(),
            ct_dic['lf'].loc[cate, 'SNP'] / ct_dic['lf']['SNP'].sum(),
            ct_dic['lf'].loc[cate, 'SNP'], ct_dic['lf']['SNP'].sum(),
        ))
    
    print('')
#     break

Consequence
missense_variant                         	 common 0.165 ( 5611/34036) 	 lf 0.188 (5946/31686)
downstream_gene_variant                  	 common 0.174 ( 5927/34036) 	 lf 0.187 (5940/31686)
synonymous_variant                       	 common 0.189 ( 6438/34036) 	 lf 0.148 (4700/31686)
upstream_gene_variant                    	 common 0.139 ( 4715/34036) 	 lf 0.148 (4677/31686)
non_coding_transcript_exon_variant       	 common 0.137 ( 4646/34036) 	 lf 0.135 (4263/31686)
intron_variant,non_coding_transcript_variant 	 common 0.054 ( 1843/34036) 	 lf 0.050 (1589/31686)
intron_variant                           	 common 0.046 ( 1562/34036) 	 lf 0.049 (1541/31686)
3_prime_UTR_variant,NMD_transcript_variant 	 common 0.027 (  914/34036) 	 lf 0.026 (815/31686)
missense_variant,NMD_transcript_variant  	 common 0.011 (  389/34036) 	 lf 0.014 (443/31686)

IMPACT
MODIFIER                                 	 common 0.602 (20476/34036) 	 lf 0.618 (19585/31686)
MODERATE                           

### Old code

In [44]:
CHR = 1
dic_mbin = {
    'common' : [0.05, 1],
    'lf' : [0.005, 0.05],
    'rare' : [0, 0.005]
}
PGEN_FILE = '/n/scratch3/users/j/jz286/imp_geno/ukb_imp_chr@_v3'
ANNOT_FILE = '/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/baseline_annot/vep'\
    '/ukb_imp_chr@_v3.vep.annot.gz'
PANNOT_FILE = '/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/protein_domain/'\
    'protein_domain_@m_@m.chr@c.pannot_mat.npz'

In [45]:
df_snp_chr = gdreg.util.read_pgen(PGEN_FILE.replace("@", "%d" % CHR))["pvar"]
df_snp_chr = df_snp_chr[["CHR", "SNP", "BP"]].copy()
df_snp_chr['MAF'] = gdreg.util.read_pgen(
    PGEN_FILE.replace("@", "%d" % CHR))["afreq"]["MAF"].astype(np.float32)
df_snp_chr.index = df_snp_chr["SNP"]   
print("PGEN n_snp=%d" % df_snp_chr.shape[0])

df_annot = gdreg.util.read_annot(ANNOT_FILE.replace("@", "%d" % CHR))

PGEN n_snp=1161341


In [46]:
dic_pannot = {}
for mbin in dic_mbin:
    if mbin == 'rare':
        dic_pannot[mbin] = np.zeros([1,1])
    else:
        dic_pannot[mbin] = gdreg.util.read_pannot_mat(PANNOT_FILE.replace('@c', '%d'%CHR).replace('@m', mbin))

In [47]:
temp_dic = {x:y for x,y in zip(df_annot['SNP'], df_annot['AN:DOMAINS'])}
df_snp_chr['AN:DOMAINS'] = [temp_dic[x] if x in temp_dic else '' for x in df_snp_chr.index]

In [48]:
dic_mbin = {
    'common' : [0.05, 1],
    'lf' : [0.005, 0.05],
    'rare' : [0, 0.005]
}

for mbin in dic_mbin: 
    maf_lb,maf_ub = dic_mbin[mbin]
    temp_df = df_snp_chr.loc[(df_snp_chr['MAF']>=maf_lb) & (df_snp_chr['MAF']<maf_ub)]
    print('%s n_snp=%d n_pd=%d ratio=%0.3f n_pair=%d' % (
        mbin, temp_df.shape[0], (temp_df['AN:DOMAINS']!='').sum(),
        (temp_df['AN:DOMAINS']!='').sum() / temp_df.shape[0], dic_pannot[mbin].sum().sum()
    ))

common n_snp=496456 n_pd=3343 ratio=0.007 n_pair=18552
lf n_snp=320940 n_pd=3337 ratio=0.010 n_pair=16524
rare n_snp=343945 n_pd=4617 ratio=0.013 n_pair=0


In [49]:
df_annot

,CHR,SNP,BP,CM,AN:DOMAINS,AN:GENE
0,1,rs147999235,738539,0,"PANTHER:PTHR12138,Prints:PR02045",ENSG00000269831
1,1,rs41285790,865628,0,"Low_complexity_(Seg):seg,PANTHER:PTHR10417:SF5...",ENSG00000268179
2,1,rs28419423,871215,0,"PANTHER:PTHR12247,PANTHER:PTHR12247:SF67",ENSG00000268179
3,1,rs564703450,877981,0,"PANTHER:PTHR12247,PANTHER:PTHR12247:SF67",ENSG00000188976
4,1,rs148327885,878331,0,"PANTHER:PTHR12247,PANTHER:PTHR12247:SF67,Low_c...",ENSG00000188976
...,...,...,...,...,...,...
11292,1,rs140745254,249150022,0,"Low_complexity_(Seg):seg,PANTHER:PTHR11389:SF4...",ENSG00000171163
11293,1,rs13313088,249150116,0,"PANTHER:PTHR11389:SF410,PANTHER:PTHR11389",ENSG00000171163
11294,1,rs34013644,249211350,0,Pfam:PF13843,ENSG00000185220
11295,1,rs41305584,249211581,0,Pfam:PF13843,ENSG00000185220


In [24]:
df_snp_chr

,CHR,SNP,BP,MAF,AN:DOMAINS
SNP,,,,,
rs587769434,22,rs587769434,16050527,0.001391,
rs62224609,22,rs62224609,16051249,0.100340,
rs62224614,22,rs62224614,16053862,0.101977,
rs7286962,22,rs7286962,16054454,0.105103,
22:16056406_C_T,22,22:16056406_C_T,16056406,0.001328,
...,...,...,...,...,...
rs370652263,22,rs370652263,51237712,0.055025,
rs541098394,22,rs541098394,51238318,0.003576,
rs553081191,22,rs553081191,51238328,0.001977,


In [4]:

df_annot = gdreg.util.read_annot(ANNOT_FILE.replace("@", "%d" % CHR))

In [5]:
df_annot

,CHR,SNP,BP,CM,AN:DOMAINS,AN:GENE
0,22,rs531101971,16449346,0,"Gene3D:1.20.1070.10,Prints:PR00245,PROSITE_pro...",ENSG00000130538
1,22,rs139948519,17072347,0,"Superfamily:SSF52029,Pfam:PF00118,Gene3D:3.50....",ENSG00000240122
2,22,rs41277596,17072411,0,"Superfamily:SSF52029,Pfam:PF00118,Gene3D:3.50....",ENSG00000240122
3,22,rs2236639,17072483,0,"Superfamily:SSF52029,Pfam:PF00118,Gene3D:3.50....",ENSG00000240122
4,22,rs145079801,17072619,0,"Superfamily:SSF52029,Pfam:PF00118,Gene3D:3.50....",ENSG00000240122
...,...,...,...,...,...,...
2771,22,rs145196448,51159798,0,"PANTHER:PTHR24135,PANTHER:PTHR24135:SF4",ENSG00000251322
2772,22,rs201483867,51160154,0,"PANTHER:PTHR24135,PANTHER:PTHR24135:SF4",ENSG00000251322
2773,22,rs749130556,51169504,0,"PANTHER:PTHR24135,PANTHER:PTHR24135:SF4,Low_co...",ENSG00000225929
2774,22,rs5771002,51183255,0,"PIRSF:PIRSF001141,PANTHER:PTHR24252",ENSG00000100312


In [6]:
df_annot_chr

,CHR,SNP,BP,MAF
SNP,,,,
rs587769434,22,rs587769434,16050527,0.001391
rs62224609,22,rs62224609,16051249,0.100340
rs62224614,22,rs62224614,16053862,0.101977
rs7286962,22,rs7286962,16054454,0.105103
22:16056406_C_T,22,22:16056406_C_T,16056406,0.001328
...,...,...,...,...
rs370652263,22,rs370652263,51237712,0.055025
rs541098394,22,rs541098394,51238318,0.003576
rs553081191,22,rs553081191,51238328,0.001977
